In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys
import os
import pandas as pd
import time
import json

project_root = ".."
sys.path.append(project_root)

from src.support_etl import *

In [ ]:
dap = pd.read_json(r'..\recetas_scrapper\dap.jsonl', lines=True)

with open("dap_leftoff.json", 'r', encoding='utf-8') as file:
    dap_leftoff = json.load(file)

print("Translating ingredients...")
for i in range(1+dap_leftoff["Position"],len(dap)):
    ingredients = dap["ingredientes"].loc[i]
    resultados = [
        " ".join(filter(None, [str(c) if c is not None else None, u, n]))
        for n, c, u in zip(ingredients['nombre'], ingredients['cantidad'], ingredients['unidad'])
    ]
    en_ingredients = [translate_es_en(e) for e in resultados]
    # print(en_ingredients)
    
    serving = dap["raciones"].loc[i]

    print("Getting nutrients...")

    nut_info, recipe_sum = get_nutrients(en_ingredients, int(serving))
    # print(dap["url"][i])
    if isinstance(nut_info, int) and nut_info != 555:
        print("Error:", nut_info) 
        break # Detenemos el proceso si hay un error que no sea el de mal parseo de la receta
    elif isinstance(nut_info, int) and nut_info == 555:
        json.dump(dap_leftoff, open("dap_leftoff.json", "w")) # Guardar la posición actual
        continue # No guardamos la receta si no se pudo parsear
    # display(nut_info)
    # print(recipe_sum)
    
    dap_leftoff["Position"] = i
    

    filtered_nut_info = nut_info[nut_info["Weight (g)"] > 0].copy() # Filtrar ingredientes con peso > 0 para evitar subir ingredientes sin peso (macros nulos)
    columns_to_normalize = filtered_nut_info.columns[2:]  # Macros

    # Normalizar los registros para que sean cada 100 g
    
    filtered_nut_info[columns_to_normalize] = (
        filtered_nut_info[columns_to_normalize]
        .div(filtered_nut_info["Weight (g)"], axis=0)
        * 100
    )

    print("Connecting to database...")
    conn = connect_db()
    recipe_id = insert_recipe(conn, dap["titulo"].loc[i], translate_es_en(dap["titulo"].loc[i]), dap["titulo"].loc[i], dap["url"].loc[i],recipe_sum.get("weight"), recipe_sum, int(serving))
    for j in filtered_nut_info.index:
        ingredient_id = get_or_create_ingredient(conn, filtered_nut_info.loc[j].get("Ingredient").lower(), filtered_nut_info.loc[j], filtered_nut_info.loc[j].get("Ingredient").lower(), translate_en_es(filtered_nut_info.loc[j].get("Ingredient")).lower())
        insert_ingredient_recipe(conn, recipe_id, ingredient_id, float(filtered_nut_info.loc[j].get("Weight (g)")))
    json.dump(dap_leftoff, open("dap_leftoff.json", "w")) # Guardar la posición actual
    time.sleep(2)



print("End")

In [3]:
def get_or_create_tag(conn, tagname):
    with conn.cursor() as cur:
    # Buscar el tag
        cur.execute("SELECT id FROM ingredients WHERE name = %s;", (tagname,))
        result = cur.fetchone()
    if result:
        # Si existe, devolver el tag
        return result[0]

In [32]:

import nltk

# nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

receta = "Lo primero es preparar la masa quebrada, para ello ponemos en un recipiente la harina, una cucharada de azúcar glas y el agua y luego, agregamos la mantequilla cortada en cubitos. Mezclamos bien hasta conseguir una masa homogénea. Al principio cuesta un poco porque la mantequilla estará dura, ayúdate de las manos para ir deshaciendo los cubos de mantequilla. Con esa masa forramos un molde de 22 cm y horneamos a 180ºC durante 10 - 15 minutos, es importante que pongas algo que pese sobre la masa, yo utilizo garbanzos pero puedes usar judías o unas bolillas de cerámica que venden para esto precisamente. Para hacer el relleno, muy sencillo, montamos las claras con una batidora de varillas, no hace falta que quede un merengue muy firme. Y luego le agregamos la harina de almendra y el azúcar. Mezclamos bien y ponemos sobre la masa quebrada previamente horneada. Volvemos a hornear durante 20-25 minutos a 180ºC. Veréis que toma un ligero tono tostado, eso es que está lista."
# receta = translate_es_en(receta)
pasos = sent_tokenize(receta) 

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\yanru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
translate_en_es("hello")

'Hola'

In [34]:
print(pasos)

['Lo primero es preparar la masa quebrada, para ello ponemos en un recipiente la harina, una cucharada de azúcar glas y el agua y luego, agregamos la mantequilla cortada en cubitos.', 'Mezclamos bien hasta conseguir una masa homogénea.', 'Al principio cuesta un poco porque la mantequilla estará dura, ayúdate de las manos para ir deshaciendo los cubos de mantequilla.', 'Con esa masa forramos un molde de 22 cm y horneamos a 180ºC durante 10 - 15 minutos, es importante que pongas algo que pese sobre la masa, yo utilizo garbanzos pero puedes usar judías o unas bolillas de cerámica que venden para esto precisamente.', 'Para hacer el relleno, muy sencillo, montamos las claras con una batidora de varillas, no hace falta que quede un merengue muy firme.', 'Y luego le agregamos la harina de almendra y el azúcar.', 'Mezclamos bien y ponemos sobre la masa quebrada previamente horneada.', 'Volvemos a hornear durante 20-25 minutos a 180ºC.', 'Veréis que toma un ligero tono tostado, eso es que está 

In [7]:
from psycopg2.errors import UniqueViolation

def insert_steps_from_jsonl(conn, jsonl, index):
    with conn.cursor() as cur:
        cur.execute("SELECT id FROM recipes WHERE url = %s;", (jsonl.loc[index, "url"],))
        try:
            recipe_id = cur.fetchone()[0]
        except:
            recipe_id = None
        # print(recipe_id)
        if recipe_id:
            try:
                steps_es = jsonl.loc[index, "instrucciones"]
                steps_en = translate_es_en(steps_es)
                # print(steps_en, "\n", steps_es)
                cur.execute("INSERT INTO steps (recipe_id, description, description_es, description_en) VALUES (%s, %s, %s, %s);", (recipe_id, steps_es, steps_en, steps_es))
                conn.commit()
            except UniqueViolation:
                print("Ya existe")


In [13]:
conn = connect_db()

dap = pd.read_json(r'..\recetas_scrapper\dap.jsonl', lines=True)

# url = dap["url"].loc[0]
for i in range(len(dap)):
    insert_steps_from_jsonl(conn, dap, i)
    break
# insert_ingredient_recipe(conn, )

203
We go over the rabbit pieces, removing any skin or fat, season them and dredge them in flour. While we do this, we put five tablespoons of olive oil in a low saucepan and brown the five unpeeled garlic cloves in it. As we flour each slice, we add it to the saucepan. Don't worry if they don't all fit. As they brown, they will shrink a little and eventually they will all fit together. We fry the slices well until they have an appetizing color and are well done inside. It will take about twenty minutes to brown them all, turning them continuously until they are like the pictures. At that point, we cut a lemon in half and squeeze it with our hand, distributing the juices over each slice. We add the glass of wine and the glass of broth and turn up the heat so that the sauce begins to reduce. Holding the handles of the pan with both hands - being careful not to burn yourself - shake the pan and make circular movements. This way the sauce will thicken while the rest of the liquid evaporat

In [29]:
def insert_steps_from_jsonl(conn, jsonl, index):
    with conn.cursor() as cur:
        cur.execute("SELECT id FROM recipes WHERE url = %s;", (jsonl.loc[index, "url"],))
        try:
            recipe_id = cur.fetchone()[0]
        except Exception as e1:
            recipe_id = None
        # print(recipe_id)
        if recipe_id:
            try:
                steps_es = jsonl.loc[index, "instrucciones"]
                steps_en = translate_es_en(steps_es)
                print(index, steps_es)
                cur.execute("INSERT INTO steps (recipe_id, description, description_es, description_en) VALUES (%s, %s, %s, %s);", (recipe_id, steps_es, steps_en, steps_es))
                conn.commit()
            except UniqueViolation:
                print("Ya existe")
            except Exception as e:
                print(e)

In [31]:
conn = connect_db()

dap = pd.read_json(r'..\recetas_scrapper\dap.jsonl', lines=True)

insert_steps_from_jsonl(conn, dap, 202)
print(dap.loc[202, "url"])

https://www.directoalpaladar.com/recetario/17-recetas-espanolas-para-viajar-salir-casa-cuarentena


In [32]:
def build_ingredients_list(ingredients_data):
    """
    Construye una lista de strings en inglés para pasárselas a EDAMAM.
    Ej: "200 gram tomatoes", "1 pinch salt", etc.
    """
    # Estructura: zip(nombre, cantidad, unidad)
    resultados = [
        " ".join(filter(
            None, 
            [str(c) if c is not None else None, u if c is not None else None, n] # La condición es para evitar un string de "None", cosa que no pasa con los que no es necesario convertir a string
        ))
        for n, c, u in zip(
            ingredients_data['nombre'], 
            ingredients_data['cantidad'], 
            ingredients_data['unidad']
        )
    ]
    en_ingredients = [translate_es_en(e) for e in resultados]
    return en_ingredients

def process_single_recipe_with_steps(df, index, db_connection_func, leftoff_path, leftoff_pos):
    """
    Procesa la receta en la posición `index` del DataFrame `dap`, 
    insertando su info nutricional, ingredientes y pasos en la BD.
    """

    # A) Obtener valores clave de la receta
    recipe_title = df["titulo"].loc[index]
    recipe_url = df["url"].loc[index]
    serving = int(df["raciones"].loc[index])
    ingredients_data = df["ingredientes"].loc[index]

    print(f"\nProcesando receta #{index+1}: {recipe_title}")

    # B) Prepara lista de ingredientes en inglés (para EDAMAM)
    en_ingredients = build_ingredients_list(ingredients_data)

    # C) Consumir la API de EDAMAM y obtener info nutricional
    nut_info, recipe_sum = get_nutrients_wrapper(en_ingredients, serving, leftoff_path, dap_leftoff, index)
    if nut_info is None or isinstance(nut_info, int):
        # Hubo error o la función devolvió un código. 
        # Salimos para avanzar a la siguiente receta.
        return

    # D) Filtrar y normalizar DataFrame de nutrientes
    filtered_nut_info = normalize_nut_info(nut_info)

    # E) Insertar en Base de Datos
    conn = db_connection_func()
    try:
        # 1) Insertar la receta
        recipe_id = insert_recipe(
            conn, 
            recipe_title,
            translate_es_en(recipe_title),
            recipe_title,
            recipe_url,
            recipe_sum.get("weight"),
            recipe_sum,
            serving
        )

        # 2) Insertar ingredientes
        insert_ingredients(conn, filtered_nut_info, recipe_id)

        # 3) Insertar pasos
        insert_steps_from_jsonl(conn, dap, index)

    finally:
        conn.close()

    # F) Actualizar `dap_leftoff` tras éxito de la inserción
    save_leftoff_position(leftoff_path, dap_leftoff, index+1)
    # Por si deseas un 'sleep' aquí
    time.sleep(1)


In [39]:
ingredients_data = dap["ingredientes"].loc[202]

In [40]:
build_ingredients_list(ingredients_data)

['1500.0 Bomba rice',
 '1.0 Free range chicken',
 '0.5 Rabbit',
 '500.0 g Flat green beans',
 '500.0 g Garrofó',
 '6.0 Optional artichoke',
 '500.0 g Snails',
 'Extra virgin olive oil',
 'Sweet paprika',
 'Crushed tomato',
 'Saffron',
 'Fresh rosemary',
 'Salt']

In [47]:
ingredients_data = dap["ingredientes"].loc[1]
ingredients = ingredients_data
print(ingredients)
[" ".join(filter(None, [str(c) if c is not None else None, u, n]))
            for n, c, u in zip(ingredients['nombre'], ingredients['cantidad'], ingredients['unidad'])]

{'nombre': ['Morcilla de arroz', 'Aceite de oliva virgen extra en spray, uno o dos toques'], 'cantidad': [1.0, None], 'unidad': [None, None]}


['1.0 Morcilla de arroz',
 'Aceite de oliva virgen extra en spray, uno o dos toques']